# Alep Tutorial

## Dispersal

Rational

In [1]:
# Python import
import collections
import numpy as np

# Wheat simulation
from alinea.adel.astk_interface import AdelWheat

# Lesion 
from alinea.alep.fungus import Lesion, Fungus

# Dispersal Models
from alinea.alep.dispersal_emission import SimpleEmission
from alinea.alep.dispersal_transport import SeptoriaRainDispersal, PowderyMildewWindDispersal, BrownRustDispersal

# Generic algorithm to disperse DispersalUnit on MTG
from alinea.alep.protocol import disperse

# Ploting functions
from alinea.alep.disease_outputs import plot3d_transparency

# PlantGL geometric library
import openalea.plantgl.all as pgl

# MTG selection
from alinea.alep.architecture import get_leaves


In [2]:
%gui qt

ERROR: Cannot activate multiple GUI eventloops


### Two generic functions

In [3]:
# utilities to place the source leaf
def is_iterable(obj):
    """ Test if object is iterable """
    return isinstance(obj, collections.Iterable)
    
def get_source_leaf_and_max_height(g, position='center', relative_height=2./3):
    """ Returns the vertex identifier and its height (z ccord).
    
    :Algorithm:
        Select all the leaves.
        Compute the bounding box of each leaf and its center (centroid).
        Compute the maximum height (z max) of all the centroids
        Then, compute the distance (norm) ???
    """
    tesselator = pgl.Tesselator()
    bbc = pgl.BBoxComputer(tesselator)
    leaves = get_leaves(g, label='LeafElement')
    centroids = g.property('centroid')
    geometries = g.property('geometry')
    targets = list(leaf for leaf in leaves if leaf in geometries.iterkeys())
    for vid in targets:
        if is_iterable(geometries[vid]):
            bbc.process(pgl.Scene(geometries[vid]))
        else:
            bbc.process(pgl.Scene([pgl.Shape(geometries[vid])]))
        center = bbc.result.getCenter()
        centroids[vid] = center
    zmax = max(centroids.items(), key=lambda x:x[1][2])[1][2]
    distances = {vid:pgl.norm(centroids[vid]-(0,0,relative_height*zmax)) for vid in centroids}
    if position=='center':
        return min(distances.items(), key=lambda x:x[1])[0], zmax
    elif position=='border':
        return max(distances.items(), key=lambda x:x[1])[0], zmax


## Create your own lesion

In [4]:
fungus_name = "lesion_tutorial"

# Create our own emitting lesion for this example
class LesionTutoDispersal(Lesion):
    def __init__(self, mutable=False):
        super(LesionTutoDispersal, self).__init__()
        self.fungus_name = fungus_name

    def emission(*args, **kwds):
        return 10000


## Create a lesion instance used for the dispersal model

In [5]:
fungus = Fungus(Lesion=LesionTutoDispersal, parameters={"name":fungus_name})
#source_lesion = fungus.lesion()


## Create a wheat architectural model with geometry at a given development stage

In [32]:
# create wheat canopy 
adel= AdelWheat(nplants=100, nsect=7)
g = adel.setup_canopy(1500)

In [33]:
#g = adel.grow_dd??

In [ ]:
g = adel.grow_dd

# Plot the 3D plants

In [7]:
pgl.Viewer.display(adel.plot(g))

# Select some leaves to set the infectious status

In [8]:
leaf_ids = get_leaves(g, label='LeafElement')

arr = np.arange(len(leaf_ids))
np.random.shuffle(arr)

nb_sporulating_leaves = 25
indices = arr[0:nb_sporulating_leaves].tolist()

#for i in indices:
    
source_leaves = [leaf_ids[i] for i in indices]
# Deterministic choice 
#leaf_index = 1
# Select randomly a leaf
#from random import randint
#leaf_index = randint(1, len(leaf_ids))-1

#source_leaf = leaf_ids[leaf_index]

# Alternatively, pick a leaf based on its position in the canopy
#vid, hmax = get_source_leaf_and_max_height(g)
#source_leaf = g.node(vid)


# Select a set of leaves of a given plant and infest some of its leaves

In [10]:
plant_ids = g.vertices(scale=1)
nb_plants = len(plant_ids)
pid = plant_ids[nb_plants / 2] 

#Compute the centroid
vid, hmax = get_source_leaf_and_max_height(g)
print hmax
max_scale = g.max_scale()
centroids = g.property('centroid')
leaves = [v for v in g.components_at_scale(pid, scale=max_scale) if 'Leaf' in g.label(v)]
def select_leaves(v):
    return centroids[v].z < hmax/3. if v in centroids else False
source_leaves = [v for v in leaves if select_leaves(v)]
source_leaves

70.6290865529


[9243, 9244, 9245, 9246, 9262, 9263, 9264, 9265]

# Inoculate the source leaves

In [17]:
# inoculate this leaf
for i in source_leaves:
    source_lesion = fungus.lesion()
    source_lesion.is_sporulating = True # Required for Popdrops usage
    source_leaf = g.node(i)
    source_leaf.lesions = [source_lesion]

# Setup the dispersal models

In [13]:
# Setup dispersal models + parameters for dispersion
emitter = SimpleEmission()
#rain 3d (slow)
transporter = SeptoriaRainDispersal()
# wind 3d (callonec model)
#transporter = PowderyMildewWindDispersal(label='LeafElement')
# wind 1D
#transporter = BrownRustDispersal(domain_area=adel.domain_area)
# missing : rian 1D (septo 3d dispersal model)


# Simulate using the generic algorithm

In [18]:
# Simulate one dispersal event
g = disperse(g, emission_model=emitter, transport_model=transporter, fungus_name=fungus_name)


# Plot the results

In [15]:
# Visualize (currently only works for septoria)
# rain/ wind
transporter.plot_distri_3d(g)
# wind 1D
#transporter.plot_layers(g)


In [23]:
gs=split(g)

In [25]:
gs[1].display()

MTG : nb_vertices=137, nb_scales=5
/MS			(id=1)											
^/metamer1			(id=2)											
^/internode			(id=3)											
^/HiddenElement			(id=4)											
^<StemElement			(id=5)											
	+sheath			(id=6)										
	^/HiddenElement			(id=7)										
	^<StemElement			(id=8)										
	^<blade			(id=9)										
	^/HiddenElement			(id=10)										
^<metamer2			(id=11)											
^/internode			(id=12)											
^/HiddenElement			(id=13)											
^<StemElement			(id=14)											
	+sheath			(id=15)										
	^/HiddenElement			(id=16)										
	^<StemElement			(id=17)										
	^<blade			(id=18)										
	^/HiddenElement			(id=19)										
^<metamer3			(id=20)											
^/internode			(id=21)											
^/HiddenElement			(id=22)											
^<StemElement			(id=23)											
	+sheath			(id=24)										
	^/HiddenElement			(id=25)										
	^<StemElement			(id=26)										
	^<blade			(id=27)										
	^/HiddenElement			(id=28)										
^<metamer4			(id=29)											
^/int